# MidTerm Summary
## Scrape data from website

In [3]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

quote_list = []
author_list = []
for i in range(1, 11):  # from 1 to 10
    url = f'http://quotes.toscrape.com/page/{i}/' # stringF
    response = requests.get(url)
    soup = bs(response.text, 'html.parser')  # bs(response.content, 'html.parser')
    # soup.head
    # print(soup.prettify())

    quotes = soup.find_all('span', itemprop='text') # itemprop and class_ are same thing
    authors = soup.find_all('small', class_='author')

    for quote in quotes:
        quote_list.append(quote.get_text())

    for author in authors:
        author_list.append(author.get_text())

    df1 = pd.DataFrame(data=quote_list, columns=['Quotes'])
    df2 = pd.DataFrame(data=author_list, columns=['Authors'])
    quotes = pd.concat([df1, df2], axis=1)  # axis=1 -> horizontally
    quotes.to_excel('quotes_week2.xlsx')

In [4]:
# Another way is to use dictionaries:
quotes_and_authors_dic = dict(zip(authors, quotes))
df_dict = pd.DataFrame(quotes_and_authors_dic, index=[0])
df_dict.to_excel('q_a_dict_excel_sheet_week2.xlsx')
from xlwt import Workbook

wb = Workbook()
sheet1 = wb.add_sheet('Sheet from the dictionary')

# this throws an error ???
# counter = 1;
# for key, value in quotes_and_authors_dic.items():
#     sheet1.write(counter, 0, key)
#     sheet1.write(counter, 1, value)
#     counter += 1
#
# wb.save('xlwt q_a_dict_excel_sheet_week2.xlsx')

## Selenium

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.get('https://www.youtube.com/c/brettconti/videos')
# driver.implicitly_wait(10)  # if the loading is done it will continue (not like time.sleep())
# videos = driver.find_elements(By.CLASS_NAME, "style-scope ytd-grid-video-renderer")
# # specify a class name that occurs multiple times and use find_elements to get a list returned
#
# video_list = []
# for video in videos:
#     title = video.find_element(By.XPATH, './/*[@id="video-title"]').text  # put a dot at front!
#     views = video.find_element(By.XPATH, './/*[@id="metadata-line"]/span[1]').text
#     posted = video.find_element(By.XPATH, './/*[@id="metadata-line"]/span[2]').text
#     video_list.append({'title': title, 'views': views, 'posted': posted})
#
# df = pd.DataFrame(video_list)
driver.quit()  # just closes the tab -> always do that!

In [6]:
from pygooglenews import GoogleNews
gn = GoogleNews()
top = gn.top_news()
top
business = gn.topic_headlines('business')
search = gn.search('%22New+York%22')  #, when = '6m')
print(search.keys())
print(search['feed'])
print(search['entries'])
#---------------------------
for item in search['entries']:
    print(item['title'])

def get_title(search):
    stories = []
    search = gn.search(search)
    for item in search['entries']:
        stories.append({'title': item.title, 'link': item.link})
    return stories

a = get_title('weather')
weather = pd.DataFrame(a)
weather['Title'] = weather['title'].str.split(' - ', expand=True)[0]
weather['Source'] = weather['title'].str.split(' - ', expand=True)[1]
weather.drop(['title'], axis=1)

weather.rename(columns={'link': 'Link'}, inplace=True)
weather = weather[['Title', 'Source', 'Link']]
weather.to_excel('weather.xlsx')

dict_keys(['feed', 'entries'])
{'generator_detail': {'name': 'NFE/5.0'}, 'generator': 'NFE/5.0', 'title': '"%22New+York%22" - Google News', 'title_detail': {'type': 'text/plain', 'language': None, 'base': '', 'value': '"%22New+York%22" - Google News'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://news.google.com/search?q=%2522New%2BYork%2522&ceid=US:en&hl=en-US&gl=US'}], 'link': 'https://news.google.com/search?q=%2522New%2BYork%2522&ceid=US:en&hl=en-US&gl=US', 'language': 'en-US', 'publisher': 'news-webmaster@google.com', 'publisher_detail': {'email': 'news-webmaster@google.com'}, 'rights': '2022 Google Inc.', 'rights_detail': {'type': 'text/plain', 'language': None, 'base': '', 'value': '2022 Google Inc.'}, 'updated': 'Tue, 18 Oct 2022 15:17:24 GMT', 'updated_parsed': time.struct_time(tm_year=2022, tm_mon=10, tm_mday=18, tm_hour=15, tm_min=17, tm_sec=24, tm_wday=1, tm_yday=291, tm_isdst=0), 'subtitle': 'Google News', 'subtitle_detail': {'type': 'text/html', 'la

## Tokenization

In [7]:
from bs4 import BeautifulSoup as bs
import re
data = requests.get("https://www.gutenberg.org/cache/epub/8001/pg8001.html")
def strip_html_tag(text): # get rid of unnecessary spaces
    soup = bs(text, 'html.parser')
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n]+', '\n', stripped_text)
    return stripped_text
clean_content = strip_html_tag(data.content)

In [8]:
#------- word tokenization ------------------------------------
import nltk
nltk.download('punkt')
cleaned_contents = nltk.word_tokenize(clean_content)
contents = list(filter(None, [re.sub(r'[^a-zA-Z]', '', a) for a in cleaned_contents]))
# filter(None, ...) means filter all the elements that are None or '' -->  so this filtered None, and everything that is not text
contents = [content.lower() for content in contents] # make everything lowercase
#------- Counter ----------------------------------------------
from collections import Counter
c = Counter(contents)
c.most_common(10)

nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
contents_without_stopwords = [word for word in contents if word not in stopwords]
c = Counter(contents_without_stopwords)
# c.most_common(10)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sonja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sonja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
#------- Sentence Tokenization --------------------------------
from nltk.corpus import gutenberg
nltk.download('gutenberg')
nltk.corpus.gutenberg.fileids()
alice = gutenberg.raw(fileids='carroll-alice.txt')
len(alice)
print(alice[0:500])

default_st = nltk.sent_tokenize
alice_sentences = default_st(text=alice)
alice_sentences = [x.replace('\n', '') for x in alice_sentences]
alice_word = nltk.word_tokenize(alice)
alice_contents = list(filter(None, [re.sub(r'[^a-zA-Z]', '', a) for a in alice_word]))
alice_contents = [word.lower() for word in alice_contents if word.lower() not in stopwords]

ac = Counter(alice_contents)
# ac.most_common(10)
new_stopwords = ['nt', 'said', 'would', 'know', 'like', 'went'] # Let's add more to stop words
for i in new_stopwords:
    stopwords.append(i)
alice_contents = [word for word in alice_contents if word not in stopwords]
ac = Counter(alice_contents)
# ac.most_common(10)

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\sonja\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


[Alice's Adventures in Wonderland by Lewis Carroll 1865]

CHAPTER I. Down the Rabbit-Hole

Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into the
book her sister was reading, but it had no pictures or conversations in
it, 'and what is the use of a book,' thought Alice 'without pictures or
conversation?'

So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy an


## POS-tagging

In [10]:
nltk.download('tagsets')
nltk.help.upenn_tagset('NN')  # this is just for getting the info what the shortcuts stand for
text = "They refuse to permit us to obtain the refuse permit"
test1 = nltk.pos_tag(text.split())  # instead of the other thing (this is only one line though)
pd.DataFrame(test1).T # why is there the T again?

In [ ]:
sentence2 = "The little yellow dog barked at the angry cat that belongs to Heidi Choi."
tokens = nltk.word_tokenize(sentence2)
tags = nltk.pos_tag(tokens)

pattern = r"""
NP: {<DT>?<JJ>*<NN>}
VBD: {<VBD>}
IN: {<IN>}
NP: {<NNP>+}
"""

NPchunker = nltk.RegexpParser(pattern)
result = NPchunker.parse(tags)
result.draw()

In [ ]:
text = nltk.Text(word.lower() for word in nltk.corpus.gutenberg.words('melville-moby_dick.txt'))
text.similar('woman')  # not similar to the meaning but similar like also a noun

In [ ]:
from nltk.corpus import gutenberg

hamlet = gutenberg.raw('shakespeare-hamlet.txt')
default_st = nltk.sent_tokenize
hamlet_sentences = default_st(text=hamlet)
hamlet_sentences = [x.replace('\n', '') for x in hamlet_sentences]
hamlet_word = nltk.word_tokenize(hamlet)
hamlet_pos = nltk.pos_tag(hamlet_word)
nouns = []
for word, pos in hamlet_pos:
    if pos in ['NN']:
        nouns.append(word)

## Stemming

In [ ]:
# ---------- Porter-Stemmer ------------------------------------------
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
porterStemmer = PorterStemmer()
words = ['plays', 'playing', 'played', 'player', 'pharmacies', 'badly']
print([porterStemmer.stem(w) for w in words])

In [ ]:
# ---------- Snowball-Stemmer ----------------------------------------
from nltk.stem.snowball import SnowballStemmer
snowballStemmer = SnowballStemmer(language='english')
print([snowballStemmer.stem(w) for w in words])

In [ ]:
# ---------- Lancaster-Stemmer ----------------------------------------
from nltk.stem import LancasterStemmer
lancasterStemmer = LancasterStemmer()
print([lancasterStemmer.stem(w) for w in words])

## Lemmatization

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

wnl = WordNetLemmatizer()
print([wnl.lemmatize(w) for w in words])  # Remember! Without the POS tag, wnl assume every word as a noun!
tagged_words = nltk.pos_tag(words)
#print([wnl.lemmatize(w[0], w[1]) for w in tagged_words]) # ask someone why this doesn't work
print(wnl.lemmatize('dogs', 'n'))
print(wnl.lemmatize('went', 'v'))
print(wnl.lemmatize('happiest', 'a'))